In [4]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html


%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from utils.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
QFN_TMP = "{WK_DIR}/workloads/complex_workload_50k_s1.sql"
QFN_TMP2 = "{WK_DIR}/workloads/workload_50k_s1.sql"

BASEDIR = "/flash1/ziniuw/data_transfer/"
#DBS = ["financial"]
DBS = ["Accidents", "employee", "Credit", "ConsumerExpenditures", "legalActs", "Basketball_men", "Hepatitis_std", 
      "Seznam", "Carcinogenesis", "ErgastF1", "ssb", "ccs", "stats"]

#DBS = ["VisualGenome", "Airline", ]

OUTPATH_TMP = "../data/zdbs/{DB}/dfs/"

QFNS = []
allsqls = []

for db in DBS:
    sqls = []
    wk_dir = os.path.join(BASEDIR, db)
    qfn = QFN_TMP.format(WK_DIR=wk_dir)
    if not os.path.exists(qfn):
        print(qfn)
    
    #assert os.path.exists(qfn)
    with open(qfn, "r") as f:
        data = f.readlines()
        
    for sql in data:
        sqls.append(sql)
    
    qfn2 = QFN_TMP2.format(WK_DIR=wk_dir)
    with open(qfn2, "r") as f:
        data = f.readlines()
    for sql in data:
        sqls.append(sql)

    allsqls.append(sqls)

In [8]:
for j, sqls in enumerate(allsqls):
    for i, sql in enumerate(sqls):
        sql = sql.replace("IN", "in")
        sql = sql.replace("In", "in")
        sql = sql.replace("ILIKE", "like")
        sql = sql.replace("LIKE", "like")
        sqls[i] = sql
    allsqls[j] = sqls

In [9]:
sqls = allsqls[0]

In [10]:
sql = sqls[18]

In [11]:
print(sql)

SELECT MAX("upravna_enota"."id_upravna_enota" + "oseba"."vozniski_staz_MM") as agg_0 FROM "oseba" LEFT OUTER JOin "upravna_enota" ON "oseba"."upravna_enota" = "upravna_enota"."id_upravna_enota" LEFT OUTER JOin "nesreca" ON "oseba"."id_nesreca" = "nesreca"."id_nesreca"  WHERE "upravna_enota"."povrsina" != 216 AND "nesreca"."oznaka_odsek_ali_ulica" like '%000%00%';



In [12]:
#opdf,exprdf = parse_sqls_single([sql], 0, COLS_TO_TABLES)
#opdf
parse_sql_preds(sql)

,op,dtype,optype,constant,input,column,opstring,is_id
0,!=,num,!=,216,"""upravna_enota""","""povrsina""","""upravna_enota"".""povrsina"" != 216",0
1,LIKE,num,3%,'%000%00%',"""nesreca""","""oznaka_odsek_ali_ulica""","""nesreca"".""oznaka_odsek_ali_ulica"" like '%000%...",0


In [1]:
import time

def parse_and_save_dfs(sqls, db):
    
    db = db.lower()
    start = time.time()
    opdf,exprdf = parse_sqls_par(sqls, {})
    print("parse sqls took: ", int(time.time()-start))
    print("Num Ops: ", len(opdf), "Num Expressions: ", len(exprdf))
    exprdf["InputCardinality"] = -1
    exprdf["RowCount"] = -1
    exprdf["Selectivity"] = -1
    
    edf = exprdf[["exprhash", "InputCardinality", "Selectivity", "RowCount"]]
    opdf = opdf.merge(edf, on="exprhash", how="inner")
    
    print(len(exprdf))
    print(len(set(exprdf["jobid"])))
    
    opdf["workload"] = db
    exprdf["workload"] = db
    opdf["db"] = db
    exprdf["db"] = db

    start = time.time()
    
        
    
    outpath = OUTPATH_TMP.format(DB=db)
    
    make_dir(outpath)
    opfn = os.path.join(outpath, "op_df.csv")

    exprfn = os.path.join(outpath, "expr_df.csv")

    if not os.path.exists(exprfn):
        opdf.to_csv(opfn, index=False)
        print("opdf to csv took: ", time.time()-start)
        exprdf.to_csv(exprfn, index=False)
    
    return exprdf, opdf

In [62]:
#exprdf, opdf = parse_and_save_dfs(allsqls[0], DBS[0])

number of sql groups:  6
finished parsing batch:  4 in :  126 fails:  0
finished parsing batch:  3 in :  127 fails:  0
finished parsing batch:  2 in :  132 fails:  0
finished parsing batch:  1 in :  136 fails:  0
finished parsing batch:  0 in :  137 fails:  0
parse sqls took:  138
Num Ops:  389130 Num Expressions:  184695
184695
98420
opdf to csv took:  2.0283141136169434


In [14]:
for i, sqls in enumerate(allsqls):
    exprdf, opdf = parse_and_save_dfs(sqls, DBS[i])

number of sql groups:  6
finished parsing batch:  2 in :  118 fails:  0
finished parsing batch:  0 in :  118 fails:  0
finished parsing batch:  4 in :  118 fails:  0
finished parsing batch:  1 in :  118 fails:  0
finished parsing batch:  3 in :  119 fails:  0
parse sqls took:  120
Num Ops:  371466 Num Expressions:  158123
158123
96169
opdf to csv took:  2.075564384460449
number of sql groups:  6
finished parsing batch:  3 in :  126 fails:  0
finished parsing batch:  4 in :  127 fails:  0
finished parsing batch:  2 in :  135 fails:  0
finished parsing batch:  0 in :  144 fails:  0
finished parsing batch:  1 in :  145 fails:  0
parse sqls took:  146
Num Ops:  493122 Num Expressions:  188300
188300
98848
opdf to csv took:  2.7431397438049316
number of sql groups:  6
finished parsing batch:  4 in :  111 fails:  0
finished parsing batch:  3 in :  111 fails:  0
finished parsing batch:  2 in :  118 fails:  0
finished parsing batch:  1 in :  124 fails:  0
finished parsing batch:  0 in :  124 f

In [56]:
# start = time.time()
# opdf,exprdf = parse_sqls_par(sqls, {})
# print("parse sqls took: ", int(time.time()-start))
# print("Num Ops: ", len(opdf), "Num Expressions: ", len(exprdf))

number of sql groups:  6
finished parsing batch:  4 in :  126 fails:  0
finished parsing batch:  3 in :  126 fails:  0
finished parsing batch:  2 in :  131 fails:  0
finished parsing batch:  1 in :  136 fails:  0
finished parsing batch:  0 in :  137 fails:  0
parse sqls took:  138
Num Ops:  389130 Num Expressions:  184695


In [21]:
# exprdf["InputCardinality"] = -1
# exprdf["RowCount"] = -1
# exprdf["Selectivity"] = -1

In [63]:
# print(len(set(opdf["exprhash"])))
# opdf.head(5)

In [64]:
# print(len(set(exprdf["exprhash"])))
# exprdf.head(5)

In [65]:
# print(len(exprdf))
# print(len(set(exprdf["jobid"])))

In [66]:
# print(len(sqls))

In [67]:
# print(len(opdf))
# edf = exprdf[["exprhash", "InputCardinality", "Selectivity", "RowCount"]]
# opdf = opdf.merge(edf, on="exprhash", how="inner")
# print(len(opdf))
# opdf.head(5)

In [68]:
# exprdf["filtersql"].values

In [69]:
# exprdf.head(5)

In [31]:
# if "stack" in WK:
#     opdf = opdf[~opdf["input"].isin(["X", "function"])] 
#     opdf["inplen"] = opdf.apply(lambda x: len(x["input"]) ,axis=1)
#     opdf = opdf[opdf["inplen"] >= 4]
   
#     exprdf = exprdf[~exprdf["input"].isin(["X", "function"])]
#     exprdf["inplen"] = exprdf.apply(lambda x: len(x["input"]) ,axis=1)
#     exprdf = exprdf[exprdf["inplen"] >= 4]

In [70]:
# opdf["workload"] = WK
# exprdf["workload"] = WK
# opdf["db"] = WK
# exprdf["db"] = WK

# import time
# start = time.time()

# opfn = os.path.join(OUTPATH, "op_df.csv")
# exprfn = os.path.join(OUTPATH, "expr_df.csv")

# opdf.to_csv(opfn, index=False)
# print("opdf to csv took: ", time.time()-start)

# exprdf.to_csv(exprfn, index=False)

In [33]:
exprdf[["unique_filter_cols", "num_likes"]].describe(percentiles=[0.9, 0.99])

,unique_filter_cols,num_likes
count,71385.000000,71385.000000
mean,1.503509,0.428605
std,0.775599,0.645617
min,1.000000,0.000000
50%,1.000000,0.000000
90%,3.000000,1.000000
99%,4.000000,3.000000
max,7.000000,6.000000


In [34]:
opdf.groupby("input")["column"].nunique().reset_index().sort_values(by="column", ascending=False).head(50)

,input,column
0,"""nesreca""",21
1,"""oseba""",17
2,"""upravna_enota""",4


In [35]:
def print_sql(sql):
    sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
    sql = sql.replace("\n", "<br>")
    sql = sql.replace(" ", "&nbsp;")
    return sql

In [36]:
HTML(print_sql(sqls[93]))

In [38]:
HTML(opdf[["like_ops", "discrete_ops", "cont_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,like_ops,discrete_ops,cont_ops
count,264162.000000,264162.000000,264162.000000
mean,0.138684,0.747715,0.113601
std,0.345617,0.434325,0.317326
min,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000
90%,1.000000,1.000000,1.000000
99%,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [39]:
df = opdf

In [40]:
discdf = df[df.discrete_ops == 1]
#discdf.groupby(["dkind"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops")

In [41]:
discdf.groupby(["dtype"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,dtype,discrete_ops
2,string,184113
1,num,13402
0,date,3


In [42]:
ldf = df[df.like_ops == 1]
ldf.groupby("dtype").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,dtype,discrete_ops
4,unknown,17371
3,short,15017
1,num,3576
0,-1,507
2,serial,164


# unique values seen in each column

In [44]:
#inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [45]:
# dff = df[df["col"] == "function"]
# dff

In [46]:
#pdf = pd.concat(allpreds)
pdf = df

In [47]:
pdf.head(5)

,op,dtype,optype,constant,input,column,opstring,is_id,jobid,discrete_ops,cont_ops,like_ops,alias,exprhash,InputCardinality,Selectivity,RowCount
0,<=,cont,cont,198,"""upravna_enota""","""povrsina""","""upravna_enota"".""povrsina"" <= 198",0,0,0,1,0,"""upravna_enota""",203807725675968861748626587656043215204328283759,-1,-1,-1
1,!=,num,!=,5597,"""upravna_enota""","""id_upravna_enota""","""upravna_enota"".""id_upravna_enota"" != 5597",0,0,1,0,0,"""upravna_enota""",203807725675968861748626587656043215204328283759,-1,-1,-1
2,<=,cont,cont,5505,"""nesreca""","""upravna_enota""","""nesreca"".""upravna_enota"" <= 5505",0,0,0,1,0,"""nesreca""",439453855335675694614837337232318969545118378133,-1,-1,-1
3,!=,string,!=,'L',"""oseba""","""poskodba""","""oseba"".""poskodba"" != 'L'",0,0,1,0,0,"""oseba""",1381158848075027180740545243550918246269537932020,-1,-1,-1
4,=,string,in,'N',"""oseba""","""povzrocitelj_ali_udelezenec""","""oseba"".""povzrocitelj_ali_udelezenec"" in ('N')",0,1,1,0,0,"""oseba""",1444199379437138945963180534488786193090014451098,-1,-1,-1
